## H2o Top Models GLMs, Auto ML: Deeplearning, ["DRF", "GBM", "XGBoost"], include_algos all

In [ ]:
!pip install h2o

In [3]:
import h2o
h2o.init()
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
import pandas as pd
import numpy as np

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.8" 2020-07-14; OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp8d8lf8d7
  JVM stdout: /tmp/tmp8d8lf8d7/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp8d8lf8d7/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_unknownUser_v2m98i
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [5]:
X = pd.read_csv('https://raw.githubusercontent.com/gjhernandezp/ConcDriversML/master/WARX.csv')
y = pd.read_csv('https://raw.githubusercontent.com/gjhernandezp/ConcDriversML/master/WARY.csv')

train = X.join(y)
print('train ', train.shape)


train_h = h2o.H2OFrame(train)
train_h["Genero"] = train_h["Genero"].asfactor()
train_h["Estrato"] = train_h["Estrato"].asfactor()
train_h["NivelEd"] = train_h["NivelEd"].asfactor()
train_h["Accidente"] = train_h["Accidente"].asfactor()

predictors = ["Genero","Edad","Estrato","NivelEd","ConcProm1","ConcProm2_1","ConcProm2_2"]
response = "Accidente"

train_h


train  (167, 8)
Parse progress: |█████████████████████████████████████████████████████████| 100%


Genero,Edad,Estrato,NivelEd,ConcProm1,ConcProm2_1,ConcProm2_2,Accidente
0,21,2,4,25.09,18.7828,31.3967,1
1,72,3,4,43.5474,39.5988,47.4956,1
0,47,1,0,40.9353,34.0787,47.7915,1
0,43,2,3,57.7271,55.1533,60.3006,1
0,33,3,4,47.5,41.1186,53.9386,1
0,68,4,5,60,54.8072,65.2324,1
0,74,2,1,71.3089,73.1998,69.4181,0
0,24,2,1,41.9732,24.3062,59.6389,1
1,24,3,4,70.1969,64.6597,75.7336,0
1,40,3,4,67.258,68.5133,66.0029,0


In [12]:
from h2o.automl import H2OAutoML

predictors = ["Genero","Edad","Estrato","NivelEd","ConcProm1","ConcProm2_1","ConcProm2_2"]
response = "Accidente"
aml = H2OAutoML(max_models=200, 
                seed=1, 
                include_algos=["DeepLearning"], 
                max_runtime_secs=60*10,
                nfolds = 5,
                keep_cross_validation_predictions=True)
aml.train(x=predictors, y=response, training_frame=train_h)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)
aml.leader.predict(train_h)

aml.leader.save_mojo()

print(aml.leader)

print("Actual")
actual = np.array(train[response])
print(actual)

pred_df = aml.leader.predict(train_h)
predicted = np.array(pred_df.as_data_frame()["predict"])
print("predicted")
print(predicted)

print("Prediction Accuracy")
count = 0
for i in range(len(predicted)): 
    if predicted[i] == actual[i]:
       count= count+1
print(count/len(predicted))

print("MSE")
np.sum(np.square(np.array(train[response])-predicted))/len(predicted)

aml.leader.varimp_plot()

AutoML progress: |████████████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid__2_AutoML_20200817_033340_model_3


Status of Neuron Layers: predicting Accidente, 2-class classification, bernoulli distribution, CrossEntropy loss, 262,502 weights/biases, 3.0 MB, 4,073 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,21,Input,20,,,,,,,,,
1,,2,500,RectifierDropout,20,0,0,0.17818,0.380044,0,-0.000791887,0.0611463,0.487848,0.0143392
2,,3,500,RectifierDropout,20,0,0,0.0222356,0.0861555,0,-0.00196885,0.0464045,0.994046,0.00770625
3,,4,2,Softmax,,0,0,0.00147458,0.00184169,0,0.000301179,0.246263,2.26238e-05,0.000840464




ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.08655138426441317
RMSE: 0.2941961662979536
LogLoss: 0.6272128936706227
Mean Per-Class Error: 0.10020554984583763
AUC: 0.9103288797533402
AUCPR: 0.9650282047724175
Gini: 0.8206577595066804

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9591709572658218: 


,,0,1,Error,Rate
0,0,19.0,9.0,0.3214,(9.0/28.0)
1,1,0.0,139.0,0.0,(0.0/139.0)
2,Total,19.0,148.0,0.0539,(9.0/167.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-08-17 03:40:00,0.000 sec,None,0.000000,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-08-17 03:40:01,2 min 52.320 sec,331 obs/sec,2.491018,1,416.0,0.228823,0.348958,0.624802,0.898510,0.951310,0.600719,0.053892
2,,2020-08-17 03:40:07,2 min 58.216 sec,349 obs/sec,14.646707,6,2446.0,0.253300,0.535574,0.540239,0.906346,0.963142,0.600719,0.053892
3,,2020-08-17 03:40:11,3 min 2.497 sec,363 obs/sec,24.389222,10,4073.0,0.294196,0.627213,0.379797,0.910329,0.965028,0.600719,0.053892



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,NivelEd.4,1.000000,1.000000,0.058423
1,NivelEd.1,0.985055,0.985055,0.057549
2,Estrato.4,0.982346,0.982346,0.057391
3,NivelEd.6,0.972765,0.972765,0.056831
4,NivelEd.3,0.958440,0.958440,0.055994
5,ConcProm1,0.957794,0.957794,0.055957
6,NivelEd.5,0.957794,0.957794,0.055957
7,ConcProm2_2,0.952787,0.952787,0.055664
8,Estrato.5,0.952703,0.952703,0.055659
9,ConcProm2_1,0.952479,0.952479,0.055646



See the whole table with table.as_data_frame()

Actual
[1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1
 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 0]
deeplearning prediction progress: |███████████████████████████████████████| 100%
predicted
[1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1
 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1]
Prediction Accuracy
0.9461077844311377
MSE


0.05389221556886228

In [13]:
from h2o.automl import H2OAutoML

predictors = ["Genero","Edad","Estrato","NivelEd","ConcProm1","ConcProm2_1","ConcProm2_2"]
response = "Accidente"
aml = H2OAutoML(max_models=200, 
                seed=1, 
                include_algos=["DRF", "GBM", "XGBoost"],
                max_runtime_secs=60*10,
                nfolds = 5,
                keep_cross_validation_predictions=True)

aml.train(x=predictors, y=response, training_frame=train_h)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

aml.leader.save_mojo()

print(aml.leader)

print("Actual")
actual = np.array(train[response])
print(actual)

pred_df = aml.leader.predict(train_h)
predicted = np.array(pred_df.as_data_frame()["predict"])
print("predicted")
print(predicted)

print("Prediction Accuracy")
count = 0
for i in range(len(predicted)): 
    if predicted[i] == actual[i]:
       count= count+1
print(count/len(predicted))

print("MSE")
np.sum(np.square(np.array(train[response])-predicted))/len(predicted)

aml.leader.varimp_plot()

AutoML progress: |█████████████████
03:44:02.279: Skipping training of model GBM_5_AutoML_20200817_034345 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20200817_034345.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 167.0.


███████████████████████████████████████| 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_grid__1_AutoML_20200817_034345_model_16


Model Summary: 


,,number_of_trees
0,,109.0




ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.048225597152892034
RMSE: 0.2196032721816595
LogLoss: 0.19790504072604131
Mean Per-Class Error: 0.09301130524152112
AUC: 0.960431654676259
AUCPR: 0.9903535763512912
Gini: 0.920863309352518

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6742227673530579: 


,,0,1,Error,Rate
0,0,22.0,6.0,0.2143,(6.0/28.0)
1,1,1.0,138.0,0.0072,(1.0/139.0)
2,Total,23.0,144.0,0.0419,(7.0/167.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-08-17 03:44:22,18.559 sec,0.0,0.500000,0.693147,0.500000,0.832335,1.000000,0.167665
1,,2020-08-17 03:44:22,18.593 sec,5.0,0.419657,0.543237,0.914568,0.967848,1.164185,0.047904
2,,2020-08-17 03:44:22,18.602 sec,10.0,0.366821,0.452736,0.925360,0.972526,1.170633,0.047904
3,,2020-08-17 03:44:22,18.621 sec,15.0,0.331353,0.393075,0.926644,0.972817,1.170633,0.047904
4,,2020-08-17 03:44:22,18.633 sec,20.0,0.308818,0.354704,0.926259,0.972735,1.170633,0.047904
5,,2020-08-17 03:44:22,18.642 sec,25.0,0.290061,0.322506,0.926387,0.972656,1.170367,0.047904
6,,2020-08-17 03:44:22,18.656 sec,30.0,0.280402,0.304156,0.918294,0.968316,1.163095,0.047904
7,,2020-08-17 03:44:22,18.669 sec,35.0,0.270083,0.286375,0.919193,0.968487,1.163095,0.047904
8,,2020-08-17 03:44:22,18.682 sec,40.0,0.262036,0.271951,0.919450,0.968543,1.163095,0.047904
9,,2020-08-17 03:44:22,18.692 sec,45.0,0.254805,0.259428,0.939363,0.978456,1.177881,0.041916



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,ConcProm2_1,44.569588,1.000000,0.482558
1,ConcProm1,40.758926,0.914501,0.441299
2,ConcProm2_2,5.082629,0.114038,0.055030
3,Edad,1.496032,0.033566,0.016198
4,NivelEd.3,0.127033,0.002850,0.001375
5,Estrato.3,0.100683,0.002259,0.001090
6,NivelEd.5,0.049222,0.001104,0.000533
7,Estrato.4,0.046488,0.001043,0.000503
8,NivelEd.0,0.039461,0.000885,0.000427
9,NivelEd.1,0.033299,0.000747,0.000361



Actual
[1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1
 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 0]
xgboost prediction progress: |████████████████████████████████████████████| 100%
predicted
[1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1
 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1]
Prediction Accuracy
0.9580838323353293
MSE


0.041916167664670656

In [6]:
from h2o.automl import H2OAutoML

predictors = ["Genero","Edad","Estrato","NivelEd","ConcProm1","ConcProm2_1","ConcProm2_2"]
response = "Accidente"
aml = H2OAutoML(max_models=800, 
                seed=1, 
                max_runtime_secs=60*10,
                nfolds = 5,
                keep_cross_validation_predictions=True)

aml.train(x=predictors, y=response, training_frame=train_h)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

aml.leader.save_mojo()

print(aml.leader)

print("Actual")
actual = np.array(train[response])
print(actual)

pred_df = aml.leader.predict(train_h)
predicted = np.array(pred_df.as_data_frame()["predict"])
print("predicted")
print(predicted)

print("Prediction Accuracy")
count = 0
for i in range(len(predicted)): 
    if predicted[i] == actual[i]:
       count= count+1
print(count/len(predicted))

print("MSE")
np.sum(np.square(np.array(train[response])-predicted))/len(predicted)

aml.leader.varimp_plot()

AutoML progress: |████████████
06:53:42.509: Skipping training of model GBM_5_AutoML_20200817_065322 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20200817_065322.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 167.0.


████████████████████████████████████████████| 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid__1_AutoML_20200817_065322_model_12


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,82.0,82.0,14455.0,1.0,5.0,4.426829,1.0,15.0,9.402439




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0037475772892849243
RMSE: 0.06121745902342668
LogLoss: 0.02971607304095537
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.794646065030722: 


,,0,1,Error,Rate
0,0,28.0,0.0,0.0,(0.0/28.0)
1,1,0.0,139.0,0.0,(0.0/139.0)
2,Total,28.0,139.0,0.0,(0.0/167.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-08-17 06:54:25,12.257 sec,0.0,0.373568,0.452164,0.500000,0.832335,1.000000,0.167665
1,,2020-08-17 06:54:25,12.266 sec,5.0,0.288948,0.294960,0.979060,0.995398,1.201439,0.029940
2,,2020-08-17 06:54:25,12.273 sec,10.0,0.250693,0.240516,0.987153,0.997219,1.201439,0.029940
3,,2020-08-17 06:54:25,12.280 sec,15.0,0.218787,0.195970,0.994090,0.998797,1.201439,0.023952
4,,2020-08-17 06:54:25,12.288 sec,20.0,0.194457,0.163361,0.995632,0.999116,1.201439,0.017964
5,,2020-08-17 06:54:25,12.296 sec,25.0,0.167242,0.128227,0.996917,0.999380,1.201439,0.017964
6,,2020-08-17 06:54:25,12.303 sec,30.0,0.152484,0.111427,0.998201,0.999639,1.201439,0.017964
7,,2020-08-17 06:54:25,12.311 sec,35.0,0.137101,0.094137,0.999486,0.999898,1.201439,0.005988
8,,2020-08-17 06:54:25,12.318 sec,40.0,0.129297,0.085184,0.999486,0.999898,1.201439,0.005988
9,,2020-08-17 06:54:25,12.326 sec,45.0,0.118454,0.074268,1.000000,1.000000,1.201439,0.000000



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,ConcProm2_1,57.818455,1.000000,0.576973
1,ConcProm1,20.947271,0.362294,0.209034
2,ConcProm2_2,10.111001,0.174875,0.100898
3,Edad,7.362508,0.127338,0.073471
4,Estrato,2.051599,0.035483,0.020473
5,NivelEd,1.619064,0.028003,0.016157
6,Genero,0.300034,0.005189,0.002994



Actual
[1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1
 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 0]
gbm prediction progress: |████████████████████████████████████████████████| 100%
predicted
[1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1
 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 0]
Prediction Accuracy
1.0
MSE


0.0

In [15]:
h2o.ls()
h2o.shutdown()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  


H2O session _sid_a53d closed.


## TPOT AutoML Models

In [9]:
# Install tpot on the server
!pip install tpot

     |████████████████████████████████| 92kB 5.5MB/s 
     |████████████████████████████████| 163kB 13.6MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11956 sha256=a673b767243cfffc8a6be0a0c5d98c65d961f5e3e0cc70610900d67461809d6b
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit


In [10]:
import pandas as pd
import numpy as np
X = pd.read_csv('https://raw.githubusercontent.com/gjhernandezp/ConcDriversML/master/WARX.csv')
y = pd.read_csv('https://raw.githubusercontent.com/gjhernandezp/ConcDriversML/master/WARY.csv')
X = np.array(X)
y = np.array(y).reshape((-1,))
print(X.shape)
print(y.shape)

(167, 7)
(167,)


In [11]:
# Import the tpot classifier 
from tpot import TPOTClassifier 

# Create a tpot object with a few parameters
tpot = TPOTClassifier(scoring = 'accuracy', 
                    max_time_mins = 240, 
                    n_jobs = -1,
                    verbosity = 2,
                    cv = 5)

In [ ]:
# Fit the tpot model on the training data
tpot.fit(X, y)
tpot_best_model = tpot.fitted_pipeline_
print(tpot_best_model)
# Export the pipeline as a python script file
tpot.export('tpot_exported_pipeline1.py')

# Show the final model
print(tpot.fitted_pipeline_)


Generation 1 - Current best internal CV score: 0.9577540106951872
Generation 2 - Current best internal CV score: 0.9577540106951872
Generation 3 - Current best internal CV score: 0.9577540106951872
Generation 4 - Current best internal CV score: 0.9577540106951872
Generation 5 - Current best internal CV score: 0.9577540106951872
Generation 6 - Current best internal CV score: 0.9577540106951872
Generation 7 - Current best internal CV score: 0.9577540106951872
Generation 8 - Current best internal CV score: 0.9577540106951872
Generation 9 - Current best internal CV score: 0.9577540106951872
Generation 10 - Current best internal CV score: 0.9577540106951872
Generation 11 - Current best internal CV score: 0.9577540106951872
Generation 12 - Current best internal CV score: 0.9577540106951872
Generation 13 - Current best internal CV score: 0.9577540106951872
Generation 14 - Current best internal CV score: 0.9577540106951872
Generation 15 - Current best internal CV score: 0.9579322638146168
Gen

In [ ]:
from sklearn.model_selection import cross_val_score
tpot_best_model = tpot.fitted_pipeline_

# Evaluate the final model 
print(tpot.score(X, y))

cross_val5 = cross_val_score(tpot_best_model, X, y, scoring='accuracy', cv=5)
print(cross_val5)
print(cross_val5.mean())

cross_val10 = cross_val_score(tpot_best_model, X, y, scoring='accuracy', cv=10)
print(cross_val10)
print(cross_val10.mean())

In [ ]:
# Fit the tpot model on the training data
tpot.fit(X, y)
tpot_best_model = tpot.fitted_pipeline_
print(tpot_best_model)
# Export the pipeline as a python script file
tpot.export('tpot_exported_pipeline2.py')

# Show the final model
print(tpot.fitted_pipeline_)

In [ ]:
from sklearn.model_selection import cross_val_score
tpot_best_model = tpot.fitted_pipeline_

# Evaluate the final model 
print(tpot.score(X, y))

cross_val5 = cross_val_score(tpot_best_model, X, y, scoring='accuracy', cv=5)
print(cross_val5)
print(cross_val5.mean())

cross_val10 = cross_val_score(tpot_best_model, X, y, scoring='accuracy', cv=10)
print(cross_val10)
print(cross_val10.mean())

In [ ]:
# Fit the tpot model on the training data
tpot.fit(X, y)
tpot_best_model = tpot.fitted_pipeline_
print(tpot_best_model)
# Export the pipeline as a python script file
tpot.export('tpot_exported_pipeline3.py')

# Show the final model
print(tpot.fitted_pipeline_)

In [ ]:
from sklearn.model_selection import cross_val_score
tpot_best_model = tpot.fitted_pipeline_

# Evaluate the final model 
print(tpot.score(X, y))

cross_val5 = cross_val_score(tpot_best_model, X, y, scoring='accuracy', cv=5)
print(cross_val5)
print(cross_val5.mean())

cross_val10 = cross_val_score(tpot_best_model, X, y, scoring='accuracy', cv=10)
print(cross_val10)
print(cross_val10.mean())